In [1]:
# Import libraries
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_preprocessing import TextPreprocessor, load_data
from feature_extraction import FeatureExtractor
from train_model import SentimentModel, prepare_train_test_split, train_multiple_models
from evaluate_model import ModelEvaluator, compare_models

import warnings
warnings.filterwarnings('ignore')

print("✅ All modules imported successfully!")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\param\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ All modules imported successfully!


## 1. Load and Preprocess Data

In [2]:
# Load raw data
df = load_data('../data/raw/coursera_reviews.csv')
df.head()

✅ Loaded 140320 reviews from ../data/raw/coursera_reviews.csv


,CourseId,Review,Label
0,2-speed-it,BOring,1
1,2-speed-it,Bravo !,5
2,2-speed-it,Very goo,5
3,2-speed-it,"Great course - I recommend it for all, especia...",5
4,2-speed-it,One of the most useful course on IT Management!,5


In [4]:
# Initialize preprocessor
preprocessor = TextPreprocessor(remove_stopwords=True, lemmatize=True)

# Preprocess data
# Adjust 'review' to your actual text column name
df = preprocessor.preprocess_dataframe(df, text_column='Review', output_column='cleaned_review')

print("\n✅ Preprocessing completed!")
print(f"Sample cleaned review:\n{df['cleaned_review'].iloc[0]}")


✅ Preprocessing completed!
Sample cleaned review:
boring


In [5]:
# Save processed data
df.to_csv('../data/processed/cleaned_reviews.csv', index=False)
print("✅ Processed data saved to data/processed/cleaned_reviews.csv")

✅ Processed data saved to data/processed/cleaned_reviews.csv


## 2. Feature Extraction (TF-IDF)

In [7]:
# Initialize feature extractor
extractor = FeatureExtractor(method='tfidf', max_features=5000, ngram_range=(1, 2))

# Extract features
X = extractor.fit_transform(df['cleaned_review'])
y = df['Label']  # Adjust to your actual label column

print(f"\nFeature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# Save vectorizer
extractor.save_vectorizer('../models/tfidf_vectorizer.pkl')

✅ Extracted 5000 features using TFIDF

Feature matrix shape: (140320, 5000)
Labels shape: (140320,)
✅ Vectorizer saved to ../models/tfidf_vectorizer.pkl


## 3. Train-Test Split

In [8]:
# Split data
X_train, X_test, y_train, y_test = prepare_train_test_split(X, y, test_size=0.2, random_state=42)


📊 Data Split:
   Training samples: 112256
   Testing samples: 28064
   Features: 5000


## 4. Train Baseline Models

### 4.1 Logistic Regression

In [10]:
# Train Logistic Regression
lr_model = SentimentModel(model_type='logistic')
lr_model.train(X_train, y_train)

# Evaluate
lr_evaluator = ModelEvaluator(lr_model.model, X_test, y_test, class_names=['Negative', 'Neutral', 'Positive'])
lr_metrics = lr_evaluator.print_metrics()
lr_evaluator.print_classification_report()


🚀 Training LOGISTIC model...
✅ Training completed!

📊 MODEL EVALUATION RESULTS
Accuracy:  0.7885
Precision: 0.7365
Recall:    0.7885
F1-Score:  0.7451

📋 CLASSIFICATION REPORT:



ValueError: Number of classes, 5, does not match size of target_names, 3. Try specifying the labels parameter

In [ ]:
# Confusion Matrix
lr_evaluator.plot_confusion_matrix(save_path='../reports/figures/confusion_matrix_lr.png')

### 4.2 Naive Bayes

In [ ]:
# Train Naive Bayes
nb_model = SentimentModel(model_type='naive_bayes')
nb_model.train(X_train, y_train)

# Evaluate
nb_evaluator = ModelEvaluator(nb_model.model, X_test, y_test, class_names=['Negative', 'Neutral', 'Positive'])
nb_metrics = nb_evaluator.print_metrics()
nb_evaluator.print_classification_report()

In [ ]:
# Confusion Matrix
nb_evaluator.plot_confusion_matrix(save_path='../reports/figures/confusion_matrix_nb.png')

### 4.3 Random Forest

In [ ]:
# Train Random Forest
rf_model = SentimentModel(model_type='random_forest')
rf_model.train(X_train, y_train)

# Evaluate
rf_evaluator = ModelEvaluator(rf_model.model, X_test, y_test, class_names=['Negative', 'Neutral', 'Positive'])
rf_metrics = rf_evaluator.print_metrics()
rf_evaluator.print_classification_report()

## 5. Compare Models

In [ ]:
# Compare all models
results = {
    'Logistic Regression': {
        'model': lr_model,
        'train_accuracy': lr_model.model.score(X_train, y_train),
        'test_accuracy': lr_metrics['accuracy']
    },
    'Naive Bayes': {
        'model': nb_model,
        'train_accuracy': nb_model.model.score(X_train, y_train),
        'test_accuracy': nb_metrics['accuracy']
    },
    'Random Forest': {
        'model': rf_model,
        'train_accuracy': rf_model.model.score(X_train, y_train),
        'test_accuracy': rf_metrics['accuracy']
    }
}

comparison = compare_models(results)

## 6. Save Best Model

In [ ]:
# Save the best performing model
best_model_name = comparison['Test Accuracy'].idxmax()
best_model = results[best_model_name]['model']

best_model.save_model('../models/best_baseline_model.pkl')
print(f"\n✅ Best model ({best_model_name}) saved!")

## 7. Test Predictions on Sample Reviews

In [ ]:
# Test on sample reviews
sample_reviews = [
    "This course is absolutely amazing! I learned so much.",
    "Terrible experience, waste of time and money.",
    "It was okay, nothing special but not bad either."
]

# Preprocess
cleaned_samples = [preprocessor.preprocess(review) for review in sample_reviews]

# Transform to features
sample_features = extractor.transform(cleaned_samples)

# Predict
predictions = best_model.predict(sample_features)
probabilities = best_model.predict_proba(sample_features)

# Display results
print("\n🔮 Sample Predictions:\n")
for i, review in enumerate(sample_reviews):
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {predictions[i]}")
    print(f"Confidence: {probabilities[i].max():.2%}")
    print("-" * 80)

## 8. Summary & Next Steps

**Results:**
- Best model: [Model name]
- Test accuracy: [X%]
- Key findings: [List]

**Next Steps:**
1. Hyperparameter tuning (Grid Search / Random Search)
2. Try different feature representations (Word2Vec, FastText)
3. Experiment with deep learning models (LSTM, BERT)
4. Implement aspect-based sentiment analysis